In [1]:
import json
import datetime as dt
import numpy as np
import pandas as pd
import sqlite3
from pathlib import Path

In [2]:
courses = [doc for doc in Path('../../data/courses').iterdir()]
rounds = [doc for doc in Path('../../data/rounds').iterdir()]

In [3]:
conn = sqlite3.connect('../../data/golf.sqlite')
c = conn.cursor()

In [4]:
c.execute("""DROP TABLE courses""")
conn.commit()

c.execute("""
    CREATE TABLE courses(
        courseId VARCHAR(20) PRIMARY KEY,
        courseName VARCHAR(100),
        courseUuid VARCHAR(100))
""")
conn.commit()

for i in range(len(courses)):
    with open(courses[i]) as f:
        data = json.load(f)
    id = data['courseId'].split('-')[0]
    name = ' '.join(courses[i].stem.split('-')[1:])
    uuid = data['courseUuid']
    course_data = (id, name, uuid)

    c.execute("""
              INSERT INTO courses
                (courseId, courseName, courseUuid)
              VALUES(?, ?, ?)
              """, course_data)
    conn.commit()

In [5]:
c.execute("""DROP TABLE holes""")
conn.commit()

c.execute("""
    CREATE TABLE holes(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        courseId VARCHAR(20),
        hole VARCHAR(20),
        teebox VARCHAR(20),
        slope INTEGER,
        rating INTEGER,
        distance_yards INTEGER,
        par INTEGER,
        hcp INTEGER,
        FOREIGN KEY(courseId) REFERENCES courses(courseId)
    )
""")
conn.commit()

for i in range(len(courses)):
    with open(courses[i]) as f:
        data = json.load(f)
    hole_data = []
    cell_lengths = data['scorecard']['holes']['cellCount']
    number_of_backTeeboxes = len(data['scorecard']['backTeeboxes']['teeboxes'])
    number_of_forwardTteeboxes = len(data['scorecard']['forwardTeeboxes']['teeboxes'])

    for teebox in range(number_of_backTeeboxes):
        for cell in range(cell_lengths):
            cell_data = {}
            cell_data['courseId'] = data['courseId'].split('-')[0]
            cell_data['hole'] = data['scorecard']['holes']['cells'][cell]['value']
            cell_data['par'] = int(data['scorecard']['backTeeboxes']['par']['cells'][cell]['value'])
            cell_data['hcp'] = data['scorecard']['backTeeboxes']['handicaps']['cells'][cell]['value']

            teebox_data = data['scorecard']['backTeeboxes']['teeboxes'][teebox]
            cell_data['teebox'] = teebox_data['color']
            cell_data['slope'] = teebox_data['slope']
            cell_data['rating'] = teebox_data['rating']
            cell_data['distance_yards'] = teebox_data['cells'][cell]['yards']
            hole_data.append(cell_data)

    cells = tuple(hole_data)
    c.executemany("""INSERT INTO holes
                  (courseId, hole, par, hcp, teebox, slope, rating, distance_yards)
                  VALUES(:courseId, :hole, :par, :hcp, :teebox, :slope, :rating, :distance_yards)""", cells)
    conn.commit()
    print(cells)

({'courseId': 'Gv15', 'hole': '1', 'par': 3, 'hcp': '9', 'teebox': 'Yellow', 'slope': 103, 'rating': 30.0, 'distance_yards': 112}, {'courseId': 'Gv15', 'hole': '2', 'par': 3, 'hcp': '6', 'teebox': 'Yellow', 'slope': 103, 'rating': 30.0, 'distance_yards': 159}, {'courseId': 'Gv15', 'hole': '3', 'par': 3, 'hcp': '8', 'teebox': 'Yellow', 'slope': 103, 'rating': 30.0, 'distance_yards': 128}, {'courseId': 'Gv15', 'hole': '4', 'par': 4, 'hcp': '1', 'teebox': 'Yellow', 'slope': 103, 'rating': 30.0, 'distance_yards': 348}, {'courseId': 'Gv15', 'hole': '5', 'par': 3, 'hcp': '7', 'teebox': 'Yellow', 'slope': 103, 'rating': 30.0, 'distance_yards': 136}, {'courseId': 'Gv15', 'hole': '6', 'par': 3, 'hcp': '3', 'teebox': 'Yellow', 'slope': 103, 'rating': 30.0, 'distance_yards': 210}, {'courseId': 'Gv15', 'hole': '7', 'par': 3, 'hcp': '5', 'teebox': 'Yellow', 'slope': 103, 'rating': 30.0, 'distance_yards': 184}, {'courseId': 'Gv15', 'hole': '8', 'par': 4, 'hcp': '2', 'teebox': 'Yellow', 'slope': 103,

In [14]:
c.execute("""DROP TABLE scores""")
conn.commit()

c.execute("""
    CREATE TABLE scores(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        courseId VARCHAR(20),
        roundId VARCHAR(20),
        playerId VARCHAR(20),
        player_name VARCHAR(100),
        start_date DATETIME,
        scoring_type VARCHAR(20),
        pace_of_play FLOAT,
        course_hcp FLOAT,
        holes INTEGER,
        strokes INTEGER,
        FOREIGN KEY(courseId) REFERENCES courses(courseId)
    )
""")
conn.commit()

for i in range(len(rounds)):
    with open(rounds[i]) as f:
        data = json.load(f)
    round_data = []
    number_of_teams = len(data['model']['game']['teams'])

    for team in range(number_of_teams):
        number_of_players = len(data['model']['game']['teams'][team]['players'])
        for player in range(number_of_players):
            number_of_holes = len(data['model']['game']['teams'][team]['players'][0]['scores'])
            for hole in range(number_of_holes):
                cell_data = {}
                cell_data['courseId'] = data['model']['detail']['golfCourseWebId'].split('-')[0]
                cell_data['roundId'] = data['roundGroupId']
                cell_data['playerId'] = data['userAccountId']
                cell_data['player_name'] = data['model']['game']['teams'][team]['players'][player]['name']
                cell_data['start_date'] = dt.datetime.strptime(\
                                            data['model']['detail']['formattedStartTime'], \
                                            '%b %d, %Y')
                pace_of_play_formatted = data['model']['detail']['formattedPaceOfPlay'].split(':')
                cell_data['pace_of_play'] = dt.timedelta( \
                                    hours=int(pace_of_play_formatted[0]), \
                                    minutes=int(pace_of_play_formatted[1])).total_seconds() // 60
                cell_data['course_hcp'] = float(data['model']['detail']['formattedCourseHandicap'])
                cell_data['holes'] = data['model']['header']['holes'][hole]
                cell_data['strokes'] = data['model']['game']['teams'][team]['players'][player]['scores'][hole]['score']


                round_data.append(cell_data)

    cells = tuple(round_data)
    c.executemany("""INSERT INTO scores
                  (courseId, roundId, playerId, player_name, course_hcp, holes, strokes, start_date, pace_of_play)
                  VALUES(:courseId, :roundId, :playerId, :player_name, :course_hcp, :holes, :strokes, :start_date, :pace_of_play)""", cells)
    conn.commit()
    print(cells)

({'courseId': 'Gv15', 'roundId': 'O649ME', 'playerId': 'xG6ggB', 'player_name': 'Jerome', 'start_date': datetime.datetime(2021, 9, 9, 0, 0), 'pace_of_play': 134.0, 'course_hcp': 25.0, 'holes': '1', 'strokes': 2}, {'courseId': 'Gv15', 'roundId': 'O649ME', 'playerId': 'xG6ggB', 'player_name': 'Jerome', 'start_date': datetime.datetime(2021, 9, 9, 0, 0), 'pace_of_play': 134.0, 'course_hcp': 25.0, 'holes': '2', 'strokes': 4}, {'courseId': 'Gv15', 'roundId': 'O649ME', 'playerId': 'xG6ggB', 'player_name': 'Jerome', 'start_date': datetime.datetime(2021, 9, 9, 0, 0), 'pace_of_play': 134.0, 'course_hcp': 25.0, 'holes': '3', 'strokes': 4}, {'courseId': 'Gv15', 'roundId': 'O649ME', 'playerId': 'xG6ggB', 'player_name': 'Jerome', 'start_date': datetime.datetime(2021, 9, 9, 0, 0), 'pace_of_play': 134.0, 'course_hcp': 25.0, 'holes': '4', 'strokes': 6}, {'courseId': 'Gv15', 'roundId': 'O649ME', 'playerId': 'xG6ggB', 'player_name': 'Jerome', 'start_date': datetime.datetime(2021, 9, 9, 0, 0), 'pace_of_pl